![header](https://i.imgur.com/I4ake6d.jpg)

# IN SITU GLOBAL SEAS TRAINING

<div style="text-align: right"><i> 13-01-Part-one-out-of-five </i></div>

***
# An overview of  the GLO `NRT` product/dataset

***
**General Note 1**: Execute each cell through the <button class="btn btn-default btn-xs"><i class="icon-play fa fa-play"></i></button> button from the top MENU (or keyboard shortcut `Shift` + `Enter`).<br>
<br><br>
**General Note 2**: If, for any reason, the kernel is not working anymore, in the top MENU, click on the <button class="btn btn-default btn-xs"><i class="fa fa-repeat icon-repeat"></i></button> button. Then, in the top MENU, click on "Cell" and select "Run All Above Selected Cell".<br>
***

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc">
    <ul class="toc-item">
        <li><span><a href="#1.-Introduction" data-toc-modified-id="1.-Introduction">1. Introduction</a></span></li>
        <li>
            <span><a href="#2.-Setup" data-toc-modified-id="2.-Setup">2. Setup</a></span>
            <ul>
                <li><span><a href="#2.1.-Python-packages" data-toc-modified-id="2.1.-Python-packages">2.1. Python packages</a></span></li>
               <li><span><a href="#2.2.-Auxiliary-functions" data-toc-modified-id="2.2-Setup">2.2. Auxiliary functions</a></span></li>
                <li><span><a href="#2.3.-Copernicus-Database" data-toc-modified-id="2.3.-Copernicus-Database">2.3. Copernicus database</a></span></li>
               <li><span><a href="#2.4.-Target-area" data-toc-modified-id="2.4-Target-area">2.4. Target area</a></span></li>
            </ul>
        </li>
        <li>
            <span><a href="#3.-COLLECTION-OVERVIEW" data-toc-modified-id="3.-COLLECTION-OVERVIEW">3. Collection overview</a></span>
            <ul>
                <li><span><a href="#3.1.-Getting-started" data-toc-modified-id="3.1.-Getting-started-packages">3.1. Getting started</a></span></li>
                <li><span><a href="#3.2.-Choosing-collection" data-toc-modified-id="3.2.-Chosing-collection">3.2. Choosing collection</a></span></li>
                <li>
                    <span><a href="#3.3.-Overview" data-toc-modified-id="3.3.-Overview">3.3. Overview</a></span>
                    <ul>
                <li><span><a href="#3.3.1.-Platforms" data-toc-modified-id="3.3.1.-Platforms">3.3.1. Platforms</a></span></li>
                <li><span><a href="#3.3.2.-Source-types-(data-types)" data-toc-modified-id="3.3.2.-Source-types-(data-types)">3.3.2. Source types (data types)</a></span></li>
                <li><span><a href="#3.3.3.-Feature-types-(file-types)" data-toc-modified-id="3.3.3.-Feature-types-(file-types)">3.3.3. Feature types (file types)</a></span></li>
                <li><span><a href="#3.3.4.-Parameters" data-toc-modified-id="3.3.4.-Parameters">3.3.4. Parameters</a></span></li>
                <li><span><a href="#3.3.5.-Providers" data-toc-modified-id="3.3.5.-Providers">3.3.5. Providers</a></span></li>
            </ul>
                </li>
            </ul>
        </li>
        <li><span><a href="#4.-Wrap-up" data-toc-modified-id="4.-Wrap-up">4. Wrap-up</a></span></li>
        <li><span><a href="#5.-Next-tutorial" data-toc-modified-id="5.-Next-tutorial">5. Next tutorial</a></span></li>
    </ul>
</div>

***

## 1. Introduction 
[Go back to the "Table of contents"](#Table-of-Contents)

This notebook focus on getting an overview of the files available in the different collections (`latest`, `monthly` and `history`) available within the In Situ Near Real Time product/dataset covering the Global Seas: `INSITU_GLO_NRT_OBSERVATIONS_013_030`. Click [here](https://resources.marine.copernicus.eu/?option=com_csw&task=results?option=com_csw&view=details&product_id=INSITU_GLO_NRT_OBSERVATIONS_013_030) to  view the dedicated section of this product within the [CMEMS Catalog](http://marine.copernicus.eu/services-portfolio/access-to-products/).

Any In Situ NRT product/dataset is just a bunch of netcdf files produced by the platforms (*drifters, profilers, gliders, moorings, HF-radars, vessels etc*) deployed in a certain seas (in this case, the Global Seas); so many files that navigation/subsetting can be challenging. 

To smooth the process of data discovery to users, the In Situ TAC provides a set of `index files` that describe the aforementioned file collections. In addition to these index files, it is also possible to find further info regarding each platform contributting with files on the `index_platform.txt`.

| Index |  Description |
| ------- | ----------- |
| `index_latest.txt`  |  List of available files within the latest collection + metadata | 
| `index_monthly.txt`   | List of available files within the monthly collection + metadata |
| `index_history.txt`   |  List of available files within the history collection + metadata |
| `index_platform.txt`   | Full list of platforms + metadata |

Check more about these files in the <a href="https://archimer.ifremer.fr/doc/00324/43494/" target="_blank">Product User Manual</a>

<div class="alert alert-block alert-success">
<b>OBJECTIVE</b>
    
***  
To discover which platforms, file types, data types, parameters and providers are to be expected in a specific collection of the BAL NRT product/dataset by using the aforementioned index files. 

## 2. Setup
### 2.1. Python packages

In [1]:
from configparser import ConfigParser
from pathlib import Path
from collections import namedtuple
import os
import warnings
warnings.filterwarnings("ignore")
import IPython
from shapely.geometry import box 
import ftputil
import pandas as pd
import folium
from folium import plugins
import json2html
from json2html import *
%matplotlib inline

In [2]:
# global parameters
config = ConfigParser()
config.read(os.path.expandvars('%USERPROFILE%\.cmems-copernicus.ini'))  #  C:\Users\{USER}
usr = config['Main']['user']
pas = config['Main']['pwd']

### 2.2. Auxiliary functions

In [90]:
module_path = Path.cwd().absolute()
if module_path not in sys.path:
    sys.path.append(module_path)
from download_copernicus_ftp import *

def spatialOverlap(row, targeted_bbox_polygon):
    """ Checks if a file contains data in the specified area (targeted_bbox_polygon)
    """
    result = False
    try:
        bounding_box = box(*[float(row[c]) for c in [
            'geospatial_lon_min', 'geospatial_lat_min', 'geospatial_lon_max', 'geospatial_lat_max']
            ])
        if targeted_bbox_polygon.intersects(bounding_box):  # check other rules on https://shapely.readthedocs.io/en/stable/manual.html
            result = True
    except Exception as e:
        pass
    return result

Directory d:\WorkData\BalticSea\_other_data\_model\Copernicus\_FTP_NRT_indexes\cmems_obs-ins_bal_phybgcwav_mynrt_na_irr/ exist: will use its files. To reload from FTP delete dir and run again.
Loading info from index_platform.txt...Replacing comma with space in
 ['Sylt', '2022-10-01T17:17:15Z', '2023-01-10T13:34:29Z', '', 'NO_TS_MO_Sylt_202203_YYYYMMDD NO_TS_MO_Sylt_202203_YYYYMM NO_TS_MO_Sylt_202203', 'State Agency for Coastal Protection & National Park and Marine Protection Schleswig-Holstein LKN.SH (LKN.SH Husum)', '5496', 'DEPH VHM0 VHM0_DM VPSP VPSP_DM VTM02 VTM02_DM VTM10 VTPK VTPK_DM VPED VPED_DM TEMP TEMP_DM', '54.91890', '8.08930', '2023-01-10T13:34:29Z']
 2086 rows
Loading info from index_latest.txt... 506/8512 rows selected
Loading info from index_monthly.txt...

Skipping line 18700: expected 12 fields, saw 14

Skipping line 2309: expected 12 fields, saw 14



 1483/18693 rows selected
Loading info from index_history.txt... 467/2427 rows selected


ValueError: too many values to unpack (expected 3)

In [4]:
index_files = ['index_latest.txt', 'index_monthly.txt', 'index_history.txt']  # files describing the content of the lastest, monthly and history netCDF file collections available within the above dataset
index_platform_file = 'index_platform.txt'  # files describing the netwotk of platforms contributting with files in the abve collections

In [5]:
def getIndexFiles(dataset, local_dir: Path):
    """
    Downloads so called `index files` - a set of files located in the CMEMS FTP server
    useful for exploring the needed product/dataset. Plase them in subdir with name of dataset or
    skip if subdir exist.
    Returns path to subdir. 
    """
    local_subdir = local_dir / dataset['name']
    if local_subdir.is_dir():
        print(f"Directory {local_subdir}/ exist: will use its files. To reload from FTP delete dir and run again.")
    else:
        #Provides
        indexes = index_files + [index_platform_file]
        with ftputil.FTPHost(dataset['host'], usr, pas) as ftp_host:  # connect to CMEMS FTP
            print(f"Downloading to {local_subdir}/: ")
            for index in indexes:
                print(index, end=', ')
                remotefile= "/".join(['Core', dataset['product'], dataset['name'], index])
                ftp_host.download(remotefile, local_subdir / index)  # remote, local
        print('Ok>')
    return local_subdir

In [24]:
def readIndexFile(path2file: Path, targeted_bbox_polygon):
    """
    Index files reader. Load as pandas dataframe the file in the provided path.
    In order to load the information contained in each of the files we will use the next function.
    """
    print('Loading info from', path2file.name, end='...')
    if path2file.name == index_platform_file:
        def replace_comma_with_space(words):
            """Correct line splitting if they are splitted by ', ': they must be splitted only if no space after ','
            """
            source_str = ','.join(words).replace(', ', ' & ')
            out = source_str.split(',')
            if out == words:
                print('Ignoring row with error (can not correct):\n', out)
                out = None
            else:
                print('Replacing comma with space in\n', out)
            return out
        
        read_csv_args = {
            'skiprows': 5,
            'delimiter': ',', 
            'engine': 'python',
            'on_bad_lines': replace_comma_with_space
        }
    else:
        read_csv_args = {
            'skiprows': 5,
            'delimiter': ',',
            'on_bad_lines': 'warn'
        }

    if targeted_bbox_polygon is None:
        result = pd.read_csv(path2file, **read_csv_args)
        try:
            result = result.rename(columns={"provider_edmo_code": "institution_edmo_code"})
        except Exception as e:
            print(e)
        print(f' {len(result)} rows')
    else:
        raw_index_info =[]
        chunks = pd.read_csv(path2file, **read_csv_args, chunksize=1000)
        n_rows = 0
        n_use = 0
        for chunk in chunks:
            chunk['spatialOverlap'] = chunk.apply(
                spatialOverlap, targeted_bbox_polygon=targeted_bbox_polygon, axis=1
            )
            chunk_use = chunk[chunk['spatialOverlap'] == True]
            n_rows += len(chunk) 
            raw_index_info.append(chunk_use)
        result = pd.concat(raw_index_info)
        print(f' {len(result)}/{n_rows} rows selected')
    return result

In [23]:
def getIndexFilesInfo(indexes_dir: Path, targeted_bbox_polygon):
    """ Loads and merges in a single entity all the information contained on each file descriptor of a given dataset
    """
    # 1) Loading the index platform info as dataframe
    indexPlatform = readIndexFile(indexes_dir / index_platform_file, None)
    indexPlatform.rename(columns={indexPlatform.columns[0]: "platform_code" }, inplace = True)
    indexPlatform = indexPlatform.drop_duplicates(subset='platform_code', keep="first")
    # 2) Loading the index files info as dataframes
    netcdf_collections = []
    for filename in index_files:
        indexFile = readIndexFile(indexes_dir / filename, targeted_bbox_polygon)
        netcdf_collections.append(indexFile)
    netcdf_collections = pd.concat(netcdf_collections)
    # 3) creating new columns: derived info
    netcdf_collections['netcdf'] = netcdf_collections['file_name'].str.split('/').str[-1]
    netcdf_collections['file_type'] = netcdf_collections['netcdf'].str.split('.').str[0].str.split('_').str[1]
    netcdf_collections['data_type'] = netcdf_collections['netcdf'].str.split('.').str[0].str.split('_').str[2]
    netcdf_collections['platform_code'] = netcdf_collections['netcdf'].str.split('.').str[0].str.split('_').str[3]
    # 4) Merging the information of all files
    headers = ['platform_code','wmo_platform_code', 'institution_edmo_code', 'last_latitude_observation', 'last_longitude_observation','last_date_observation']
    result = pd.merge(netcdf_collections, indexPlatform[headers], on='platform_code')
    print('Ok>')
    return result

### 2.3. Copernicus Database

In [31]:
localpath = Path(r'd:\WorkData\BalticSea\_other_data\_model\Copernicus')

Load the info defining product/dataset.
(In case you want to explore any other In Situ NRT product/dataset just set the above definitions accordingly and you will be able to reproduce the analysis we will perform next):

In [9]:
dataset = {
    'host': 'nrt.cmems-du.eu',  # ftp host => nrt.cmems-du.eu for Near Real Time products
    # Names of the In Situ Near Real Time product and dataset available in the above In Situ Near Real Time product:
    # Variant1: 
    #  'product': 'INSITU_BAL_NRT_OBSERVATIONS_013_032'
    #  'name': 'bal_multiparameter_nrt',
    # Variant2:
    'product': 'INSITU_BAL_PHYBGCWAV_DISCRETE_MYNRT_013_032', 
    'name': 'cmems_obs-ins_bal_phybgcwav_mynrt_na_irr',  
    # not exist: cmems_obs-ins_bal_phybgcwav_mynrt_profile
}

### 2.4. Target area


In [10]:
# Bounding box of interest
Axes2d = namedtuple('axes2d', ('x', 'y'))   # x and y are shortcats to longitude and latitude correspondingly
MinMax = namedtuple('MinMax', ('min', 'max'))
bbox = Axes2d(
    MinMax(22, 30),  # longitude limits: (min(x), max(x))
    MinMax(59, 60.5)  # latitude limits: (min(y), max(y))
)
targeted_bbox_polygon = box(bbox.x.min, bbox.y.min, bbox.x.max, bbox.y.max)

Let's see it on a map

In [12]:
folium_polygon = folium.vector_layers.Polygon(locations=[(y,x) for (x,y) in targeted_bbox_polygon.exterior.coords])
m = folium.Map(location=[bbox.y.min, bbox.x.min], zoom_start=6, min_zoom=4)
m.add_child(folium_polygon)
m.fit_bounds(folium_polygon.get_bounds())
m

## 3. COLLECTION OVERVIEW
### 3.1. Getting started

In [13]:
# Download the most recent version of the index files:
indexes_dir = getIndexFiles(dataset, localpath / '_FTP_NRT_indexes')

Directory d:\WorkData\BalticSea\_other_data\_model\Copernicus\_FTP_NRT_indexes\cmems_obs-ins_bal_phybgcwav_mynrt_na_irr/ exist: will use its files. To reload from FTP delete dir and run again.


Load indexes into a dataframe:

In [26]:
info = getIndexFilesInfo(indexes_dir, targeted_bbox_polygon)

Loading info from index_platform.txt...Replacing comma with space in
 ['Sylt', '2022-10-01T17:17:15Z', '2023-01-10T13:34:29Z', '', 'NO_TS_MO_Sylt_202203_YYYYMMDD NO_TS_MO_Sylt_202203_YYYYMM NO_TS_MO_Sylt_202203', 'State Agency for Coastal Protection & National Park and Marine Protection Schleswig-Holstein LKN.SH (LKN.SH Husum)', '5496', 'DEPH VHM0 VHM0_DM VPSP VPSP_DM VTM02 VTM02_DM VTM10 VTPK VTPK_DM VPED VPED_DM TEMP TEMP_DM', '54.91890', '8.08930', '2023-01-10T13:34:29Z']
 2086 rows
Loading info from index_latest.txt... 506/8512 rows selected
Loading info from index_monthly.txt...

Skipping line 18700: expected 12 fields, saw 14



 1483/18693 rows selected
Loading info from index_history.txt...

Skipping line 2309: expected 12 fields, saw 14



 467/2427 rows selected
Ok>


In [21]:
info.transpose()  # Shows information just loaded above

,0,1,2,3,4,5,6,7,8,9,...,2442,2443,2444,2445,2446,2447,2448,2449,2450,2451
# product_id,COP-BO-01,COP-BO-01,COP-BO-01,COP-BO-01,COP-BO-01,COP-BO-01,COP-BO-01,COP-BO-01,COP-BO-01,COP-BO-01,...,COP-BO-01,COP-BO-01,COP-BO-01,COP-BO-01,COP-BO-01,COP-BO-01,COP-BO-01,COP-BO-01,COP-BO-01,COP-BO-01
file_name,ftp://nrt.cmems-du.eu/Core/INSITU_BAL_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_BAL_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_BAL_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_BAL_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_BAL_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_BAL_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_BAL_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_BAL_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_BAL_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_BAL_PHYBGCWA...,...,ftp://nrt.cmems-du.eu/Core/INSITU_BAL_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_BAL_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_BAL_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_BAL_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_BAL_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_BAL_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_BAL_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_BAL_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_BAL_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_BAL_PHYBGCWA...
geospatial_lat_min,59.1574,59.8095,59.8095,59.3213,58.8282,59.17,59.0525,59.1335,59.1632,59.1609,...,34.37,12.25,26.0,-27.25,0.933,-34.26,-73.15,37.25,-34.4,-59.88333
geospatial_lat_max,59.8096,59.8096,59.8096,59.8096,59.8212,59.8103,59.8203,59.8103,59.8203,59.8103,...,72.42,60.6166,74.0,59.49,69.5,69.03,74.57,74.38333,70.96667,68.0
geospatial_lon_min,20.0662,22.9031,22.9031,21.0042,17.9781,20.1285,19.4297,19.9353,20.0938,20.0927,...,-17.5,-78.1833,-44.983,-95.58,-79.92,-85.28,0.01667,-12.81667,-74.55,-180.0
geospatial_lon_max,23.0736,22.9032,22.9032,23.0531,23.0596,23.0707,23.0751,23.0728,23.0773,23.0812,...,34.0,63.9833,54.0,23.52,34.6,60.7333,33.88,41.38333,163.14999,180.0
time_coverage_start,2022-12-30T00:00:00Z,2022-12-31T02:00:00Z,2023-01-01T00:00:00Z,2023-01-02T00:00:00Z,2023-01-04T00:00:00Z,2023-01-06T00:00:00Z,2023-01-10T00:00:00Z,2023-01-13T00:00:00Z,2023-01-17T00:00:00Z,2023-01-19T00:00:00Z,...,1950-11-17T14:30:00Z,1974-03-10T14:00:00Z,1990-01-16T12:15:00Z,1991-05-25T16:59:00Z,1942-01-30T09:00:00Z,1969-12-22T02:30:00Z,1947-08-26T13:00:00Z,1953-04-18T23:59:59Z,1967-08-27T18:24:00Z,1944-01-21T10:00:00Z
time_coverage_end,2022-12-30T23:50:00Z,2022-12-31T23:50:00Z,2023-01-01T23:50:00Z,2023-01-02T23:50:00Z,2023-01-04T23:50:00Z,2023-01-06T23:50:00Z,2023-01-10T23:50:00Z,2023-01-13T23:50:00Z,2023-01-17T23:50:00Z,2023-01-19T23:50:00Z,...,1958-10-22T08:30:00Z,2007-05-08T16:34:12Z,1993-06-22T11:46:48Z,1998-06-04T04:33:00Z,1975-12-20T16:00:00Z,1997-01-09T00:00:00Z,2000-12-11T15:09:00Z,1980-09-17T08:00:00Z,1980-01-12T08:00:00Z,1991-12-18T23:42:00Z
institution,STE,STE,STE,STE,STE,STE,STE,STE,STE,STE,...,Unknown institution,Unknown institution,Unknown institution,United states (Unknown),Unknown institution,Unknown institution,Unknown institution,SHOM Service Hydrographique et Oceanographiqu...,IRD Institut de recherche pour le developpeme...,SHOM Service Hydrographique et Oceanographiqu...
date_update,2023-01-04T19:01:09Z,2023-01-04T19:01:10Z,2023-01-04T19:01:10Z,2023-01-04T19:01:10Z,2023-01-05T13:00:40Z,2023-01-07T13:00:40Z,2023-01-16T07:00:57Z,2023-01-14T13:00:42Z,2023-01-18T19:00:46Z,2023-01-20T13:00:47Z,...,2021-12-20T08:40:14Z,2021-12-20T08:55:47Z,2021-12-20T10:03:36Z,2021-12-19T05:04:56Z,2021-12-20T14:02:51Z,2021-12-20T14:02:51Z,2022-02-14T07:55:13Z,2022-05-15T01:14:07Z,2022-05-14T23:32:43Z,2022-05-14T22:54:58Z


Above you can see, on the left hand side, a number of metadata fields that describes the files available in each collection (latest, monthly and history) of the product, such as: ftplink to download it (file_name), geospatial bounding box (geospatial_lat/lon_min/max), temporal coverage (time_coverage_start/end) etc
You can also see a dedicated column for each file with its corresponding values for the aforementioned descriptive metadata fields.

We will use all this information next to have an glimpse to the content of one of the product/dataset netCDF files collections.

### 3.2. Choosing collection

`Set in the next cell the collection` you wanna get an overview of and `run it` afterwards. <br> We have selected the 'latest' by default but please feel free to change it:

In [27]:
targeted_collection = 'monthly'  # latest, monthly or history
targeted_collection_info = info[info['file_name'].str.contains(targeted_collection)]
targeted_collection_info.transpose()

,13,14,15,16,17,18,19,52,53,54,...,2092,2095,2096,2097,2098,2099,2100,2101,2102,2105
# product_id,COP-BO-01,COP-BO-01,COP-BO-01,COP-BO-01,COP-BO-01,COP-BO-01,COP-BO-01,COP-BO-01,COP-BO-01,COP-BO-01,...,COP-BO-01,COP-BO-01,COP-BO-01,COP-BO-01,COP-BO-01,COP-BO-01,COP-BO-01,COP-BO-01,COP-BO-01,COP-BO-01
file_name,ftp://nrt.cmems-du.eu/Core/INSITU_BAL_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_BAL_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_BAL_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_BAL_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_BAL_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_BAL_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_BAL_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_BAL_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_BAL_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_BAL_PHYBGCWA...,...,ftp://nrt.cmems-du.eu/Core/INSITU_BAL_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_BAL_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_BAL_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_BAL_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_BAL_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_BAL_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_BAL_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_BAL_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_BAL_PHYBGCWA...,ftp://nrt.cmems-du.eu/Core/INSITU_BAL_PHYBGCWA...
geospatial_lat_min,58.8244,58.6761,58.7827,57.3969,57.3961,58.82,58.8177,59.7572,59.7572,59.7572,...,49.5333,33.05,5.2833,46.3,11.55,5.15,5.2,23.3833,5.15,53.7144
geospatial_lat_max,59.8224,59.8224,59.8224,59.8224,59.8224,59.8224,59.8224,59.7572,59.7572,59.7572,...,60.1667,59.5333,60.15,60.1833,60.1667,60.1667,60.2,60.1667,60.1667,63.98493
geospatial_lon_min,17.9781,17.8686,17.9781,17.9781,17.9759,17.9781,17.9781,23.2196,23.2196,23.2196,...,-1.9167,-11.3833,-52.5667,-11.3833,-45.8667,-52.6667,-52.6167,-32.6333,-52.6333,6.36975
geospatial_lon_max,23.0851,23.0895,23.0845,23.0865,23.0847,23.0883,23.0893,23.2196,23.2196,23.2196,...,29.0833,22.9167,29.4667,29.25,29.4,29.5833,29.5167,28.9167,29.4,22.37553
time_coverage_start,2022-05-18T07:50:00Z,2022-06-01T00:00:00Z,2022-07-01T00:00:00Z,2022-08-01T00:00:00Z,2022-09-01T00:00:00Z,2022-10-01T00:00:00Z,2022-11-01T01:00:00Z,2017-01-01T08:00:00Z,2017-02-01T00:00:00Z,2017-03-01T00:00:00Z,...,2018-04-18T22:21:00Z,2017-08-01T01:47:00Z,2018-10-01T01:46:00Z,2018-11-15T21:45:00Z,2019-07-02T15:55:00Z,2019-11-01T00:00:00Z,2020-10-01T01:56:00Z,2021-06-01T00:10:00Z,2021-09-01T01:59:00Z,2021-03-01T00:05:00Z
time_coverage_end,2022-05-31T23:50:00Z,2022-06-30T23:50:00Z,2022-07-31T21:50:00Z,2022-08-31T23:50:00Z,2022-09-30T23:00:00Z,2022-10-31T23:50:00Z,2022-11-30T23:50:00Z,2017-01-31T23:00:00Z,2017-02-28T23:00:00Z,2017-03-31T23:00:00Z,...,2018-04-30T22:19:00Z,2017-08-28T21:47:00Z,2018-10-26T03:46:00Z,2018-11-30T23:47:00Z,2019-07-31T23:59:00Z,2019-11-30T22:02:00Z,2020-10-31T23:57:00Z,2021-06-30T23:56:00Z,2021-09-30T21:59:00Z,2021-03-31T23:59:00Z
institution,STE,STE,STE,STE,STE,STE,STE,FMI,FMI,FMI,...,IRD Institut de recherche pour le developpeme...,IRD Institut de recherche pour le developpeme...,IRD Institut de recherche pour le developpeme...,IRD Institut de recherche pour le developpeme...,IRD Institut de recherche pour le developpeme...,IRD Institut de recherche pour le developpeme...,IRD Institut de recherche pour le developpeme...,IRD Institut de recherche pour le developpeme...,IRD Institut de recherche pour le developpeme...,Leibniz Institute for Baltic Sea Research Warn...
date_update,2022-08-16T13:00:40Z,2022-07-01T13:01:28Z,2022-08-01T19:01:11Z,2022-09-01T13:01:12Z,2022-10-01T13:01:18Z,2022-11-07T07:01:00Z,2022-12-05T07:01:03Z,2022-05-18T13:08:48Z,2022-05-18T13:09:26Z,2022-05-18T13:10:04Z,...,2021-11-17T16:51:57Z,2021-10-18T01:59:15Z,2021-10-18T01:59:51Z,2021-10-18T01:59:52Z,2021-10-19T11:15:00Z,2021-11-02T17:00:00Z,2021-11-21T14:50:57Z,2021-07-01T19:46:21Z,2021-10-01T23:49:19Z,2022-01-26T15:03:03Z


<div class="alert alert-block alert-success">
<b>IMPORTANT</b>

***
Please remember available options:
<ul>
    <li><i>latest</i>: daily files from platforms (last 30 days of data)</li>
    <li><i>monthly</i>: monthly files from platforms (last 5 years of data)</li>
    <li><i>history</i>: one file per platform (all platform data)</li>
</ul>

### 3.3. Overview

#### 3.3.1. Platforms

Any Situ NRT products/datasets collections are composed by files reported by a network of platforms located in the area.

Let's to know how many platforms are contributting to this product/dataset collection with files:

In [28]:
len(targeted_collection_info.groupby('platform_code').groups)

58

Let's get a closer look to this list of platforms:

In [29]:
data_platforms = [{
    'code':code, 
    'provider(s)': files['institution_edmo_code'].iloc[0],
    'files': {
        'nfiles': len(files), 
        'feature(s)': [{
            'code': ftype,
            'nfiles': len(files.groupby('file_type').get_group(ftype)),
            'sources(s)': [{
                'code': dtype,
                'nfiles': len(files.groupby('file_type').get_group(ftype).groupby('data_type').get_group(dtype)),
                'parameters': [param for param in sum([
                    files.groupby('file_type').get_group(ftype).groupby('data_type').get_group(dtype)['parameters']
                        .unique().tolist()], [])[0].split(' ') if '_' not in param and param not in ['DEPH', 'PRES', '']]
                } for dtype in files[files['file_type']==ftype]['data_type'].unique().tolist()]
            } for ftype in files['file_type'].unique().tolist()]
        }
    } for code,files in targeted_collection_info.groupby(['platform_code'])]

In [30]:
IPython.display.HTML(json2html.convert(data_platforms))

Too much info? Let's see one in particular! `Run the next cells`:
<a id='platform_selected'></a>

In [72]:
platform_code = 'DBBT'

In [73]:
selection = [platform for platform in data_platforms if platform['code'] == platform_code]
IPython.display.HTML(json2html.convert(selection))

So far we can see that each platform:
* Can have one or more providers - identified by a blank-separated list of one or more codes: the so called *edmo code* (we will see more about this later).
* Produces different type of files (see above `TS` *type* bigram) from one or more data sources (see above `MO` *source* bigram) and containing more or less parameters (see *parameters* codes). We will see next more about this!

Ok but...where is it located? There are some geographical references that we can use to infer the location of the platform:
* the bounding box of the produced files (this is, the spatial coverage - a rectangular simplification actually - of the data measured by the platform) 
* the last position reported.

Let's draw all them on a map to get an idea!:

In [84]:
#initializing map
m = folium.Map(location=[39.0, 0], zoom_start=4)
#Loooping over each file produced by the platform above
for idx, file in targeted_collection_info[targeted_collection_info['platform_code'] == platform_code].iterrows():
    #Bounding-box of each file to map as polygon
    bounding_box = box(*file[[f'geospatial_{c}' for c in ['lat_min', 'lon_min', 'lat_max', 'lon_max']]])
    folium_polygon = list(bounding_box.exterior.coords)  # [upper_left, upper_right, lower_right, lower_left]
    print
    m.add_child(folium.vector_layers.Polygon(locations=folium_polygon, popup='file bbox: ' + file['netcdf']))
#  to map as marker
print(f'Last reported', file['platform_code'], 'position:\n', file[['last_latitude_observation', 'last_longitude_observation']])
m.add_child(folium.Marker(file[['last_latitude_observation', 'last_longitude_observation']], popup=file['platform_code'] + ' last position' ))
# Zooming closer
m.fit_bounds(folium_polygon, max_zoom=6)
m

Last reported DBBT position:
 last_latitude_observation    -27.0385
last_longitude_observation   -75.7186
Name: 2105, dtype: object


On the next tutorial we will see how to use this very same geographical info to select only those files covering a specific area.

<div class="alert alert-block alert-warning">
<b>WARNING</b>
    
***  
**Don't you see the map?** Please open the notebook in a different browser (i.e Chrome!) as some have problems rendering folium maps.

#### 3.3.2. Source types (data types)

`Data sources` represents the elements that report oceanographic data. <br>So far the In Situ TAC distinguishes:

| code | description |
| ------- | ------- |
|BO | Bottle |
|BA | Data from Bathy messages on GTS |
|DB | Drifting buoys |
|DC | Drifting buoy reporting calculated sea water current |
|FB | FerryBox |
|MO | Fixed buoys or moorings |
|TG | Tide gauges |
|GL | Gliders |
|ML | Mini logger |
|CT | Oceanographic CTD profiles | //historic index file el data_type es "oceanographic CTD profiles"
|PF | Profiling floats vertical profiles |
|RE | Recopesca |
|RF | River flows |
|SF | Scanfish profiles |
|TS | Thermosalinograph data |
|XB | XBT or XCTD profiles |
|TE | Data from TESAC messages on GTS |
|SM | Sea-mammals |
|HF | High Frequency Radar |
|SD | Saildrone |
|VA | Vessel mounted ADCP |
|XX | Unknown |

Ok, this is the full list but; which ones are actually available in this product/dataset/collection?<br> to discover them:

In [48]:
targeted_collection_info['data_type'].unique().tolist()

['FB', 'MO', 'TG', 'BO', 'CT', 'GL', 'PF', 'TS']

And, How many of each?

In [49]:
data_types = [{
    'code':code,
    'platforms': {'nplatforms': len(files.groupby('platform_code')),'files': {'nfiles': len(files), 'features(s)': [{'code':ftype, 'nfiles': len(files.groupby('data_type').get_group(code).groupby('file_type').get_group(ftype)), 'nplatforms': len(files.groupby('data_type').get_group(code).groupby('platform_code'))} for ftype in files[files['data_type']==code]['file_type'].unique().tolist()]}}
} for code,files in targeted_collection_info.groupby(['data_type'])]

In [50]:
IPython.display.HTML(json2html.convert(data_types))

Too much info? Choose just one!:

In [51]:
code = 'MO'

In [52]:
selection = [dtype for dtype in data_types if dtype['code'] == code]
IPython.display.HTML(json2html.convert(selection))

#### 3.3.3. Feature types (file types)

`Feature types` represents the shape of the data reported by a given platform data source. <br>So far the In Situ TAC distinguishes 4 feature types: 

| code | description |
| ------- | ------- |
|TS | Time Serie or trajectories |
|PR | Profile |
|TV | Grided data - High Frequency Radar total velocity fields |
|RV | Grided data - High Frequency Radar radial velocity fields |
|WS | Wave spectra |

Ok, this is the full list but; which ones are actually available in this product/dataset/collection?

In [54]:
targeted_collection_info['file_type'].unique().tolist()

['TS', 'PR']

And, How many of each?

In [56]:
file_types = [{
    'code':code,
    'platforms': {'nplatforms': len(files.groupby('platform_code')),
                  'files': {'nfiles': len(files), 
                            'sources(s)': [{'code':dtype, 'nfiles': len(files.groupby('file_type').get_group(code).groupby('data_type').get_group(dtype)),
                                            'nplatforms': len(files.groupby('file_type').get_group(code).groupby('data_type').get_group(dtype).groupby('platform_code'))
                                           } for dtype in files[files['file_type']==code]['data_type'].unique().tolist()]}}
    } for code,files in targeted_collection_info.groupby(['file_type'])]

In [57]:
IPython.display.HTML(json2html.convert(file_types))

#### 3.3.4. Parameters
`parameters` represents the oceanographic variables measured by the platform and therefore present in its files. <br>So Far the In Situ TAC distinguishes the next ones (zoom with the -|+ controls if you do not see it properly):

In [58]:
IPython.display.IFrame('https://archimer.ifremer.fr/doc/00422/53381/72333.pdf', width='100%', height=350)

SyntaxError: invalid syntax (3491058713.py, line 2)

<div class="alert alert-block alert-info">
<b>TIP</b>
To find the updated catalog of parameters please check <a href="https://archimer.ifremer.fr/doc/00422/53381/">https://archimer.ifremer.fr/doc/00422/53381/</a>.

<div class="alert alert-block alert-success">
<b style="margin-right: 2em">CHALLENGE</b> Do you find in there the feature types of <a href="#platform_selected">the platform</a> we were looking at in the platform section?

*** 
In the end the platform we are checking is a profiler reporting:
* Temperature, Significant Wave Height, Average period highest 1/3 wave, Wave scalar spectral density etc

Ok, this is the full list but; which ones are actually available in this product/dataset collection?and discover it!:

In [62]:
parameter_list = [param for param in list(set(sum([parameter.split(' ') for parameter in targeted_collection_info['parameters'].tolist()],[]))) if '_' not in param and param not in ['DEPH', 'PRES', '']]
parameter_list

['WSPD',
 'FLU2',
 'DIRECTION',
 'DOX2',
 'CPHL',
 'LGHT',
 'VPSP',
 'TUR4',
 'OSAT',
 'DOXY',
 'CNDC',
 'NTRZ',
 'PHOS',
 'HCSP',
 'PHPH',
 'BBP700',
 'SSJT',
 'AMON',
 'HCDT',
 'PSAL',
 'TEMP',
 'VEMH',
 'DRYT',
 'RELH',
 'PHYC',
 'SLEV',
 'ATMP',
 'SLCA',
 'DOX1',
 'VPED',
 'NTOT',
 'WDIR',
 'VTPK',
 'SVEL',
 'VHM0',
 'CDOM']

And, How many of each?

In [63]:
parameters = [{
    'code':code,
    'platforms':{
        'nplatforms': len(targeted_collection_info[targeted_collection_info['parameters'].str.contains(code)].groupby('platform_code')),
        'files':{
            'nfiles': len(targeted_collection_info[targeted_collection_info['parameters'].str.contains(code)]), 
            'feature(s)': [{'code': ftype, 'nplatforms': len(targeted_collection_info[targeted_collection_info['parameters'].str.contains(code)].groupby('file_type').get_group(ftype).groupby('platform_code')), 'nfiles':len(targeted_collection_info[targeted_collection_info['parameters'].str.contains(code)].groupby('file_type').get_group(ftype)),'sources(s)': [{'code': dtype, 'nplatforms' : len(targeted_collection_info[targeted_collection_info['parameters'].str.contains(code)].groupby('file_type').get_group(ftype).groupby('data_type').get_group(dtype).groupby('platform_code')) ,'nfiles': len(targeted_collection_info[targeted_collection_info['parameters'].str.contains(code)].groupby('file_type').get_group(ftype).groupby('data_type').get_group(dtype))} for dtype in targeted_collection_info[targeted_collection_info['parameters'].str.contains(code)][targeted_collection_info['file_type']==ftype]['data_type'].unique().tolist()]} for ftype in targeted_collection_info[targeted_collection_info['parameters'].str.contains(code)]['file_type'].unique().tolist()]
        }
    }
} for code in parameter_list]

In [64]:
IPython.display.HTML(json2html.convert(parameters))

Too much info? Choose one! `Run the next cells`...

In [65]:
code = 'PSAL'

In [66]:
selection = [param for param in parameters if param['code'] == code]
IPython.display.HTML(json2html.convert(selection))

#### 3.3.5. Providers

Any Situ NRT products/datasets collections are composed by the files reported by a network of platforms located in the area. It is interesting always to know who are procuring these platforms and to aknowledge them!

Let's to know how many *known* providers are contributting to this product/dataset collection with files:

In [67]:
providers_list = list(set(sum([str(provider).split(' ') for provider in targeted_collection_info['institution_edmo_code'].tolist() if str(provider).isdigit()],[])))
providers_list

['1218',
 '1725',
 '545',
 '3400',
 '1850',
 '195',
 '100',
 '1051',
 '713',
 '2710',
 '440',
 '1104',
 '911']

Not clear at all, right? These numbers are identifiers provided by [SeaDatanet](https://edmo.seadatanet.org/search). <br>
Just check who is who by using SeaDataNet services:!

In [69]:
code = '1725' #change it by any other!
IPython.display.IFrame('https://edmo.seadatanet.org/report/'+code, width='100%', height=350)

How much are they contributting to the product/datasetcollection?

In [85]:
targeted_collection_info['institution_edmo_code'] = ' ' +targeted_collection_info['institution_edmo_code'].str.strip()+ ' '

In [86]:
providers = [{
    'code':code,
    'platforms':{
        'nplatforms': len(targeted_collection_info[targeted_collection_info['institution_edmo_code'].str.contains(' '+code+' ', na=False)].groupby('platform_code').groups.keys()),
        'files':{'nfiles': len(targeted_collection_info[targeted_collection_info['institution_edmo_code'].str.contains(' '+code+' ', na=False)]), 
        'feature(s)': [{'code': ftype, 'nplatforms':len(targeted_collection_info[targeted_collection_info['institution_edmo_code'].str.contains(' '+code+' ', na=False)].groupby('file_type').get_group(ftype).groupby('platform_code')), 'nfiles':len(targeted_collection_info[targeted_collection_info['institution_edmo_code'].str.contains(' '+code+' ', na=False)].groupby('file_type').get_group(ftype)),'sources(s)': [{'code': dtype, 'nplatforms': len(targeted_collection_info[targeted_collection_info['institution_edmo_code'].str.contains(' '+code+' ', na=False)].groupby('file_type').get_group(ftype).groupby('data_type').get_group(dtype).groupby('platform_code')), 'nfiles': len(targeted_collection_info[targeted_collection_info['institution_edmo_code'].str.contains(' '+code+' ', na=False)].groupby('file_type').get_group(ftype).groupby('data_type').get_group(dtype))} for dtype in targeted_collection_info[targeted_collection_info['institution_edmo_code'].str.contains(' '+code+' ', na=False)][targeted_collection_info['file_type']==ftype]['data_type'].unique().tolist()]} for ftype in targeted_collection_info[targeted_collection_info['institution_edmo_code'].str.contains(' '+code+' ', na=False)]['file_type'].unique().tolist()]}
    }
} for code in providers_list]

In [87]:
IPython.display.HTML(json2html.convert(providers))

Too much info? Choose one!

In [88]:
code = '1725'

In [89]:
selection = [provider for provider in providers if provider['code'] == code]
IPython.display.HTML(json2html.convert(selection))

<div class="alert alert-block alert-success">
<b style="margin-right: 2em">IMPORTANT</b>
    
*** 
**Are you a provider?**
If you have oe or more platforms (or know about some) producing data in the area and your institution edmo_code does not appear on the list please **contact us** by emailing the corresponding Regional CMEMS In Situ TAC Service Desk [here](http://www.marineinsitu.eu/submit-data/) **or let the speaker know**! <br>**We are interested in integrating your data!** :)

---

# 4. Wrap-up
[Go back to the "Table of Contents"](#Table-of-Contents)

So far you should already know what to expect about this product/dataset/collection: platform network, file types, source types, parameters and providers.<br> `If you don't please ask us! It is the moment!`
<br>In the next tutorial we will see how to retrieve the files comming from a certain platform, source, provider, matching a certain type of file or/and containing certain parameter. Ready? Let's go!

---

# 5. Next tutorial
[Go back to the "Table of Contents"](#Table-of-Contents)

_Click on the hyperlink below to continue the training_

[**13-02-NearRealTtime-product-subsetting-download.ipynb**](13-02-NearRealTtime-product-subsetting-download.ipynb)